In [1]:
import numpy as np
import matplotlib.pyplot as plt
from numpy import genfromtxt
import random
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torch.distributions import Normal as norm
from termcolor import colored
from sklearn.decomposition import PCA
from scipy import linalg as LA
from scipy.stats import multivariate_normal
from scipy.stats import norm
from scipy.stats import multinomial
from scipy.stats import logistic
from Main_functions import *
from itertools import permutations

In [2]:
type_of_pre_process = 'center_scaling'

In [3]:
# path_for_projected_data = 'hydrogen-combustion-3D-autoencoder-projection-of-state-space.csv'
path_for_actual_data = 'RawData/hydrogen-combustion-state-space.csv'
M_path = 'Matrices/hydrogen-combustion-3D-autoencoder-basis.csv'
mixture_fraction_path = 'RawData/hydrogen-combustion-mixture-fraction.csv'

In [4]:
# path_for_projected_data = 'syngas-combustion-3D-autoencoder-projection-of-state-space.csv'

# path_for_actual_data = 'RawData/syngas-combustion-state-space.csv'
# M_path = 'Matrices/syngas-combustion-3D-autoencoder-basis.csv'
# mixture_fraction_path = 'RawData/syngas-combustion-mixture-fraction.csv'

In [5]:
X_raw = genfromtxt(path_for_actual_data, delimiter = ',')
mixture_fractions = genfromtxt(mixture_fraction_path, delimiter = ',')
X_raw.shape, mixture_fractions.shape

((30000, 9), (30000,))

In [6]:
X_raw[10000], mixture_fractions[10000]

(array([2.22699981e+03, 2.80194937e-04, 5.74208493e-02, 2.04368036e-06,
        3.65208097e-04, 2.40747110e-01, 8.23686066e-07, 1.01962996e-09,
        1.02021563e-09]),
 0.08466189136909588)

In [7]:
def pre_process(Type = 'normalizing'):
    
    X_new = (X_raw - X_raw.mean(axis = 0))
    
    if Type == 'normalizing':
        std = X_new.std(axis = 0)
        for i in range(len(std)):
            if std[i] > 1e-3:
                X_new[:, i] /= std[i]
        
    elif Type == 'center_scaling':
        Max = X_new.max(axis = 0)
        Min = X_new.min(axis = 0)
        for i in range(len(Max)):
            if Max[i] - Min[i] > 1e-3:
                X_new[:, i]  = (X_new[:, i] - Min[i])/(Max[i]-Min[i])
            else: X_new[:, i]  = (X_new[:, i] - Min[i])
                
    elif Type == 'no_normalizing': return X_raw
                
    return X_new

In [8]:
X_processed = pre_process(Type = type_of_pre_process)
n, d = X_processed.shape
X_processed.shape

(30000, 9)

In [9]:
mixture_fractions.min(), mixture_fractions.max()

(0.0, 1.0)

In [10]:
# D = (mixture_fractions>= 0.33)
D = (mixture_fractions>= 0.15)
D.sum()

15200

In [11]:
N = 30000
n_trn = 15000
I = random.choices(np.arange(n), k = N)

In [12]:
X_processed_I = X_processed[I]
X_processed_J = np.zeros(X_processed[I].shape)
D_I = D[I]
D_I.sum()/D_I.shape

array([0.51013333])

### Test train split

In [13]:
X = torch.tensor(X_processed_I, dtype = torch.float64)
Y = torch.tensor(X_processed_J, dtype = torch.float64)
D = torch.tensor(D_I, dtype = torch.torch.int64)

In [14]:
X_T = X[:n_trn,:]
Y_T = Y[:n_trn,:]
D_T = D[:n_trn]

X_test = X[n_trn:,:]
Y_test = Y[n_trn:,:]
D_test = D[n_trn:]

In [15]:
X.shape, Y.shape, D.shape

(torch.Size([30000, 9]), torch.Size([30000, 9]), torch.Size([30000]))

### train

In [16]:
k = d
n_labels = 2

In [17]:
model = ML(d, k, n_labels, 
           X_T, Y_T, D_T, D_T, 
           X_test, Y_test, D_test, D_test)

In [18]:
model.train(learning_rate = 5e-1, #4e-1, decay = 0.9 accuracy = 73.75% is the best so far
            n_iters = 30001, 
            decay = .9)

Starting Tau:  tensor([0.8201], dtype=torch.float64, requires_grad=True)
epoch 0: loss = 0.7871000008795709
train accuracy with noise 0.4852
train accuracy without noise 0.4852
test accuracy with noise 0.4945333333333333
test accuracy without noise 0.4945333333333333
epoch 1:
 norm of B.grad = 1.7215131796273463e-12,
 B.grad.max = 7.64537994790864e-07,
 loss = 0.7871000008795709
tensor([0.7156], dtype=torch.float64, requires_grad=True)
train accuracy with noise 0.4852
train accuracy without noise 0.4852
test accuracy with noise 0.4945333333333333
test accuracy without noise 0.4945333333333333
epoch 5001:
 norm of B.grad = 4.610258356420841e-06,
 B.grad.max = 0.001626426280280686,
 loss = 0.030107658169938212
tensor([10.1804], dtype=torch.float64, requires_grad=True)
train accuracy with noise 0.9974
train accuracy without noise 0.9974
test accuracy with noise 0.9970666666666667
test accuracy without noise 0.9970666666666667
epoch 10001:
 norm of B.grad = 1.7565038793380547e-06,
 B.grad.

In [19]:
M_hat = (model.B @ model.B.T).detach().numpy()

In [20]:
model.Tau[0]

tensor(15.7403, dtype=torch.float64, grad_fn=<SelectBackward0>)

In [21]:
M_hat_normal = model.B @ model.B.T / model.Tau[0]

In [22]:
np.savetxt("B_hat_normal_hydrogen_1.csv", 
           model.B.detach().numpy()/np.sqrt(model.Tau[0].item()),
           delimiter =", ", 
           fmt ='% s')

In [23]:
U_hat, S_hat, V_hat = LA.svd(M_hat_normal.detach().numpy(), full_matrices=False)

In [24]:
S_hat

array([4.01896662e+01, 6.74059676e-13, 2.48427979e-13, 1.55530348e-13,
       2.56059186e-14, 7.94528629e-16, 1.38570901e-16, 3.32441735e-17,
       3.34542782e-18])

### Pair distance

In [25]:
X_processed = pre_process(Type = type_of_pre_process)
N, d = X_processed.shape
X_processed.shape
N, d

(30000, 9)

In [26]:
# perm = permutations([i for i in range(n)])

In [27]:
Range = [i for i in range(n)]

In [28]:
random.shuffle(Range)

In [29]:
Range[:10]

[22124, 24177, 18415, 12936, 24687, 451, 5321, 28816, 18141, 18987]

In [30]:
I = [Range[2*i] for i in range(N//2)]
J = [Range[2*i+1] for i in range(N//2)]

In [31]:
# N = 30000
# n_trn = 15000
# I = random.choices(np.arange(n), k = N)
# J = random.choices(np.arange(n), k = N)

In [32]:
X_processed_I = X_processed[I]
X_processed_J = X_processed[J]
D_I = mixture_fractions[I]
D_J = mixture_fractions[J]

In [33]:
D_diff = np.abs(D_I - D_J)

In [34]:
D_diff.mean()

0.270066493290085

In [35]:
D = (D_diff >= 0.2)
D.sum()

7254

In [36]:
n_trn = 10000

In [37]:
X = torch.tensor(X_processed_I, dtype = torch.float64)
Y = torch.tensor(X_processed_J, dtype = torch.float64)
D = torch.tensor(D, dtype = torch.torch.int64)

X_T = X[:n_trn,:]
Y_T = Y[:n_trn,:]
D_T = D[:n_trn]

X_test = X[n_trn:,:]
Y_test = Y[n_trn:,:]
D_test = D[n_trn:]

In [38]:
k = 3
n_labels = 2

In [39]:
model = ML(d, k, n_labels, 
           X_T, Y_T, D_T, D_T, 
           X_test, Y_test, D_test, D_test)

In [40]:
model.train(learning_rate = 5e-1, #4e-1, decay = 0.9 accuracy = 73.75% is the best so far
            n_iters = 30001, 
            decay = .9)

Starting Tau:  tensor([0.5080], dtype=torch.float64, requires_grad=True)
epoch 0: loss = 0.7160210483136028
train accuracy with noise 0.5178
train accuracy without noise 0.5178
test accuracy with noise 0.5136
test accuracy without noise 0.5136
epoch 1:
 norm of B.grad = 1.828148713298446e-13,
 B.grad.max = 2.6589487598606944e-07,
 loss = 0.7160210483136028
tensor([0.4547], dtype=torch.float64, requires_grad=True)
train accuracy with noise 0.5178
train accuracy without noise 0.5178
test accuracy with noise 0.5136
test accuracy without noise 0.5136
epoch 5001:
 norm of B.grad = 9.783603188247391e-07,
 B.grad.max = 0.0007939880324709032,
 loss = 0.023567172207458235
tensor([10.5399], dtype=torch.float64, requires_grad=True)
train accuracy with noise 0.9972
train accuracy without noise 0.9972
test accuracy with noise 0.9974
test accuracy without noise 0.9974
epoch 10001:
 norm of B.grad = 3.5446803554166025e-07,
 B.grad.max = 0.0004827819470484274,
 loss = 0.018832512379482403
tensor([13.2

In [41]:
M_hat = (model.B @ model.B.T).detach().numpy()

In [42]:
t = model.Tau[0].item()

In [43]:
M_hat_normal = model.B @ model.B.T / model.Tau[0]

In [44]:
np.savetxt("B_hat_normal_hydrogen_2.csv", 
           model.B.detach().numpy()/np.sqrt(model.Tau[0].item()),
           delimiter =", ", 
           fmt ='% s')

In [45]:
U_hat, S_hat, V_hat = LA.svd(M_hat_normal.detach().numpy(), full_matrices=False)

In [46]:
S_hat

array([1.46515024e+01, 5.67054607e-13, 9.17032207e-14, 1.14443500e-15,
       2.43618164e-16, 3.11493634e-17, 9.66401092e-18, 3.60552444e-18,
       1.64271950e-19])

### Without normalizing

In [47]:
X_processed = pre_process(Type = 'no_normalizing')
N, d = X_processed.shape
X_processed.shape
N, d

(30000, 9)

In [48]:
Range = [i for i in range(n)]

In [49]:
random.shuffle(Range)

In [50]:
I = [Range[2*i] for i in range(N//2)]
J = [Range[2*i+1] for i in range(N//2)]

In [51]:
X_processed_I = X_processed[I]
X_processed_J = X_processed[J]
D_I = mixture_fractions[I]
D_J = mixture_fractions[J]

In [52]:
D_diff = np.abs(D_I - D_J)

In [53]:
D_diff.mean()

0.27288783532884875

In [54]:
D = (D_diff >= 0.2)
D.sum()

7341

In [55]:
n_trn = 10000

In [56]:
X = torch.tensor(X_processed_I, dtype = torch.float64)
Y = torch.tensor(X_processed_J, dtype = torch.float64)
D = torch.tensor(D, dtype = torch.torch.int64)

X_T = X[:n_trn,:]
Y_T = Y[:n_trn,:]
D_T = D[:n_trn]

X_test = X[n_trn:,:]
Y_test = Y[n_trn:,:]
D_test = D[n_trn:]

In [57]:
k = d
n_labels = 2

In [58]:
model = ML(d, k, n_labels, 
           X_T, Y_T, D_T, D_T, 
           X_test, Y_test, D_test, D_test)

In [59]:
model.train(learning_rate = 5e-8, #4e-1, decay = 0.9 accuracy = 73.75% is the best so far
            n_iters = 30001, 
            decay = .9)

Starting Tau:  tensor([0.1030], dtype=torch.float64, requires_grad=True)
epoch 0: loss = 0.693306519043722
train accuracy with noise 0.5113
train accuracy without noise 0.5113
test accuracy with noise 0.5092
test accuracy without noise 0.5092
epoch 1:
 norm of B.grad = 1.1906852803417347,
 B.grad.max = 0.6287875510156503,
 loss = 0.693306519043722
tensor([0.1030], dtype=torch.float64, requires_grad=True)
train accuracy with noise 0.5113
train accuracy without noise 0.5113
test accuracy with noise 0.5092
test accuracy without noise 0.5092
epoch 5001:
 norm of B.grad = 0.009227351187506784,
 B.grad.max = 0.047838432158720995,
 loss = 0.641592935001861
tensor([0.1030], dtype=torch.float64, requires_grad=True)
train accuracy with noise 0.7132
train accuracy without noise 0.7132
test accuracy with noise 0.7074
test accuracy without noise 0.7074
epoch 10001:
 norm of B.grad = 0.009227492459473081,
 B.grad.max = 0.04783795410657208,
 loss = 0.6415895928853288
tensor([0.1030], dtype=torch.floa